## 1단계: 필수 패키지 설치 및 데이터 로드

In [ ]:
!pip install --upgrade azure-ai-evaluation
!pip install pandas numpy matplotlib seaborn

In [1]:
import json
import pandas as pd
import numpy as np
from typing import Dict, List
import re
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
with open('data/sample_qa.json', 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

df = pd.DataFrame(qa_data)

print(f"✓ 데이터 로드 완료: {len(df)} 개 Q&A 쌍")
print(f"\n에이전트별 분포:")
print(df['agent'].value_counts())

✓ 데이터 로드 완료: 34 개 Q&A 쌍

에이전트별 분포:
agent
product-agent          10
hallucination-agent     7
review-agent            6
policy-agent            6
rag-fail-agent          5
Name: count, dtype: int64


## 2단계: Code-based Evaluator 구현

### 예제 1: 응답 길이 평가자 (AnswerLengthEvaluator)

가장 간단한 형태의 code-based evaluator입니다.

In [2]:
class AnswerLengthEvaluator:
    """
    응답 길이를 평가하는 Code-based Evaluator
    
    Azure AI Foundry 공식 패턴:
    - __call__ 메서드 구현 (호출 가능한 클래스)
    - 키워드 인자(**kwargs) 사용
    - Dict 형식으로 결과 반환
    """
    
    def __init__(self):
        pass
    
    def __call__(self, *, response: str, **kwargs) -> Dict:
        """
        응답 길이를 평가합니다.
        
        Args:
            response: 평가할 응답 텍스트
            **kwargs: 추가 파라미터 (무시됨)
        
        Returns:
            Dict: {"answer_length": int}
        """
        return {"answer_length": len(response)}

print("✓ AnswerLengthEvaluator 정의 완료")

✓ AnswerLengthEvaluator 정의 완료


In [3]:
# AnswerLengthEvaluator 테스트
answer_length_evaluator = AnswerLengthEvaluator()
result = answer_length_evaluator(response="빛의 속도는 초당 약 30만 킬로미터입니다.")
print(f"평가 결과: {result}")

평가 결과: {'answer_length': 24}


## 3단계: 고급 Code-based Evaluator 구현

### 예제 2: 응답 품질 평가자 (ResponseQualityEvaluator)

여러 메트릭을 종합하여 응답 품질을 평가합니다.

In [4]:
class ResponseQualityEvaluator:
    """
    응답 품질을 다각도로 평가하는 Code-based Evaluator
    
    평가 항목:
    1. 응답 길이 (length_score)
    2. 구조화 정도 (structure_score)
    3. 키워드 포함도 (keyword_score)
    4. 전체 품질 점수 (quality_score)
    """
    
    def __init__(self):
        # 중요 키워드 정의
        self.important_keywords = [
            r'(예|예시|example)',
            r'(단계|step)',
            r'(참고|note|주의)',
            r'(방법|how to)',
            r'(추천|recommend)'
        ]
    
    def _evaluate_length(self, response: str) -> float:
        """응답 길이 평가 (0.0 ~ 1.0)"""
        length = len(response)
        if length < 50:
            return 0.3
        elif length < 100:
            return 0.6
        elif length <= 500:
            return 1.0
        elif length <= 1000:
            return 0.9
        else:
            return 0.6
    
    def _evaluate_structure(self, response: str) -> float:
        """구조화 정도 평가 (0.0 ~ 1.0)"""
        score = 0.0
        
        # 문장 구분 (마침표)
        if response.count('.') >= 2:
            score += 0.3
        
        # 단락 구분 (줄바꿈)
        if response.count('\n') >= 1:
            score += 0.3
        
        # 리스트 또는 구조화 요소
        if any(char in response for char in [':', '-', '•', '1.', '2.']):
            score += 0.4
        
        return min(score, 1.0)
    
    def _evaluate_keywords(self, response: str) -> float:
        """키워드 포함도 평가 (0.0 ~ 1.0)"""
        found_count = sum(
            1 for pattern in self.important_keywords
            if re.search(pattern, response, re.IGNORECASE)
        )
        return min(found_count * 0.25, 1.0)
    
    def __call__(self, *, response: str, **kwargs) -> Dict:
        """
        응답 품질을 종합 평가합니다.
        
        Returns:
            Dict: {
                "length_score": float,
                "structure_score": float,
                "keyword_score": float,
                "quality_score": float,
                "quality_reason": str
            }
        """
        length_score = self._evaluate_length(response)
        structure_score = self._evaluate_structure(response)
        keyword_score = self._evaluate_keywords(response)
        
        # 가중치 적용 (길이 30%, 구조 40%, 키워드 30%)
        quality_score = (
            length_score * 0.3 +
            structure_score * 0.4 +
            keyword_score * 0.3
        )
        
        # 등급 결정
        if quality_score >= 0.8:
            reason = "우수 - 적절한 길이와 구조를 갖춤"
        elif quality_score >= 0.6:
            reason = "양호 - 기본 요건 충족"
        elif quality_score >= 0.4:
            reason = "보통 - 개선 필요"
        else:
            reason = "미흡 - 대폭 개선 필요"
        
        return {
            "length_score": round(length_score, 2),
            "structure_score": round(structure_score, 2),
            "keyword_score": round(keyword_score, 2),
            "quality_score": round(quality_score, 2),
            "quality_reason": reason
        }

print("✓ ResponseQualityEvaluator 정의 완료")

✓ ResponseQualityEvaluator 정의 완료


In [5]:
# ResponseQualityEvaluator 테스트
quality_evaluator = ResponseQualityEvaluator()

test_response = """Azure AI Foundry에서 평가를 수행하는 방법은 다음과 같습니다:

1. 평가자를 선택하거나 정의합니다 (예: QAEvaluator, 사용자 지정 평가자)
2. 평가 데이터를 준비합니다
3. 평가를 실행하고 결과를 분석합니다

참고: Azure OpenAI 모델 설정이 필요합니다."""

result = quality_evaluator(response=test_response)
print("=== 평가 결과 ===")
print(f"길이 점수: {result['length_score']}")
print(f"구조 점수: {result['structure_score']}")
print(f"키워드 점수: {result['keyword_score']}")
print(f"종합 품질 점수: {result['quality_score']}")
print(f"평가 이유: {result['quality_reason']}")

=== 평가 결과 ===
길이 점수: 1.0
구조 점수: 1.0
키워드 점수: 0.75
종합 품질 점수: 0.92
평가 이유: 우수 - 적절한 길이와 구조를 갖춤


## 4단계: 전체 데이터셋 평가

### 4-1. 응답 길이 평가

In [6]:
print("=== 응답 길이 평가 시작 ===")
print(f"대상: {len(df)} 개 Q&A 쌍\n")

length_results = []

for i, row in df.iterrows():
    result = answer_length_evaluator(response=row['response'])
    result['agent'] = row['agent']
    length_results.append(result)
    
    if (i + 1) % 10 == 0:
        print(f"  [{i + 1}/{len(df)}] 완료")

length_df = pd.DataFrame(length_results)
print(f"\n✓ 평가 완료: {len(length_results)}개")
print(f"\n평균 응답 길이: {length_df['answer_length'].mean():.1f}자")
print(f"최대 응답 길이: {length_df['answer_length'].max()}자")
print(f"최소 응답 길이: {length_df['answer_length'].min()}자")

=== 응답 길이 평가 시작 ===
대상: 34 개 Q&A 쌍

  [10/34] 완료
  [20/34] 완료
  [30/34] 완료

✓ 평가 완료: 34개

평균 응답 길이: 79.3자
최대 응답 길이: 132자
최소 응답 길이: 27자


### 4-2. 응답 품질 평가

In [7]:
print("=== 응답 품질 평가 시작 ===")
print(f"대상: {len(df)} 개 Q&A 쌍\n")

quality_results = []

for i, row in df.iterrows():
    result = quality_evaluator(response=row['response'])
    result['agent'] = row['agent']
    quality_results.append(result)
    
    if (i + 1) % 10 == 0:
        print(f"  [{i + 1}/{len(df)}] 완료")

quality_df = pd.DataFrame(quality_results)

print(f"\n✓ 평가 완료: {len(quality_results)}개")
print(f"\n평균 품질 점수: {quality_df['quality_score'].mean():.2f}")

# 전체 결과 표시
print("\n=== 평가 결과 ===\n")
from IPython.display import display
pd.set_option('display.max_rows', None)
display(quality_df)

=== 응답 품질 평가 시작 ===
대상: 34 개 Q&A 쌍

  [10/34] 완료
  [20/34] 완료
  [30/34] 완료

✓ 평가 완료: 34개

평균 품질 점수: 0.30

=== 평가 결과 ===



,length_score,structure_score,keyword_score,quality_score,quality_reason,agent
0,0.6,0.3,0.0,0.30,미흡 - 대폭 개선 필요,product-agent
1,0.6,0.3,0.0,0.30,미흡 - 대폭 개선 필요,product-agent
2,0.6,0.3,0.0,0.30,미흡 - 대폭 개선 필요,product-agent
3,0.3,0.3,0.0,0.21,미흡 - 대폭 개선 필요,product-agent
4,0.3,0.0,0.0,0.09,미흡 - 대폭 개선 필요,product-agent
5,0.6,0.3,0.0,0.30,미흡 - 대폭 개선 필요,product-agent
6,0.3,0.0,0.0,0.09,미흡 - 대폭 개선 필요,product-agent
7,1.0,0.3,0.0,0.42,보통 - 개선 필요,review-agent
8,1.0,0.3,0.0,0.42,보통 - 개선 필요,review-agent
9,0.3,0.3,0.0,0.21,미흡 - 대폭 개선 필요,review-agent


## 6단계: Prompty-based Evaluator 구현

Prompty-based evaluator는 `.prompty` 파일을 사용하여 LLM 기반 평가를 수행합니다.

**구조:**
- `evaluators/friendliness.prompty` - 프롬프트 정의 파일
- Python 코드로 로드하여 사용

**참고:** `promptflow` 패키지와 Azure OpenAI 연결이 필요하며, LLM 호출 비용이 발생합니다.

In [ ]:
# ============================================================
# model_config 설정 (Prompty-based Evaluator용)
# ============================================================
model_config = {
    "azure_endpoint": "https://your-endpoint.openai.azure.com/",
    "api_key": "your-api-key",
    "azure_deployment": "gpt-4",
    "api_version": "2024-12-01-preview"
}



In [ ]:
# Prompty-based Evaluator 구현
from promptflow.core import Prompty, AzureOpenAIModelConfiguration
import json
import os
from typing import Dict, Any

class ResponseQualityPrompyEvaluator:
    """
    Prompty 파일을 사용하여 응답 품질을 평가하는 Evaluator
    
    Azure AI Foundry의 Prompty를 사용하여 LLM 기반 평가를 수행합니다.
    """
    
    def __init__(self, model_config: Dict = None):
        """
        Args:
            model_config: Azure OpenAI 모델 설정
                {
                    "azure_endpoint": "https://your-endpoint.openai.azure.com/",
                    "api_key": "your-api-key",
                    "azure_deployment": "gpt-4",
                    "api_version": "2024-12-01-preview"
                }
        """
        self.model_config = model_config
        self.prompty_path = "prompt/response_quality.prompty"
        
        # 환경 변수 설정 (Prompty가 필요로 함)
        os.environ['AZURE_OPENAI_ENDPOINT'] = model_config['azure_endpoint']
        os.environ['AZURE_OPENAI_API_KEY'] = model_config['api_key']
        os.environ['AZURE_OPENAI_DEPLOYMENT'] = model_config['azure_deployment']
        os.environ['AZURE_OPENAI_API_VERSION'] = model_config.get('api_version', '2024-12-01-preview')
        
        # Prompty 로드
        self._prompty = Prompty.load(source=self.prompty_path)
        
        print(f"✓ Prompty 로드 완료: {self.prompty_path}")
        print(f"  모델: {model_config.get('azure_deployment', 'N/A')}")
        print(f"  엔드포인트: {model_config.get('azure_endpoint', 'N/A')[:50]}...")
    
    def __call__(self, *, question: str, response: str, context: str = "", ground_truth: str = "", **kwargs) -> Dict:
        """
        응답의 품질을 평가합니다.
        
        Args:
            question: 질문
            response: 평가할 응답 텍스트
            context: 참고 정보
            ground_truth: 정답
            **kwargs: 추가 파라미터
        
        Returns:
            Dict: {
                "accuracy_score": float,       # 1~5점
                "completeness_score": float,   # 1~5점
                "relevance_score": float,      # 1~5점
                "clarity_score": float,        # 1~5점
                "overall_score": float,        # 1~5점
                "reasoning": str               # 평가 근거
            }
        """
        # Prompty 실행 (LLM 호출)
        result = self._prompty(
            question=question,
            response=response,
            context=context,
            ground_truth=ground_truth
        )
        
        # JSON 파싱
        if isinstance(result, str):
            # 결과가 문자열인 경우 JSON 파싱
            return json.loads(result)
        else:
            # 이미 딕셔너리인 경우 그대로 반환
            return result

print("✓ ResponseQualityPrompyEvaluator 클래스 정의 완료")

✓ ResponseQualityPrompyEvaluator 클래스 정의 완료


In [11]:
# ResponseQualityPrompyEvaluator 초기화
print("=== ResponseQualityPrompyEvaluator 초기화 ===")
prompty_evaluator = ResponseQualityPrompyEvaluator(model_config=model_config)
print("\n✓ 평가자 준비 완료\n")

=== ResponseQualityPrompyEvaluator 초기화 ===
✓ Prompty 로드 완료: evaluators/response_quality.prompty
  모델: gpt-4.1-mini
  엔드포인트: https://architecture-test-foundry.cognitiveservice...

✓ 평가자 준비 완료



In [13]:
# 전체 데이터셋 평가
print("=== 전체 데이터셋 Prompty Evaluator 평가 ===")
print(f"대상: {len(df)}개 Q&A 쌍\n")
print("⚠️ 실제 LLM 호출이 발생합니다:")
print(f"  - 예상 비용: 약 ${len(df) * 0.001:.2f} (예상치)")
print(f"  - 예상 시간: 약 {len(df) * 3 // 60}~{len(df) * 5 // 60}분")
print(f"\n진행 중...\n")

prompty_results = []
error_count = 0

for idx, row in df.iterrows():
    try:
        result = prompty_evaluator(
            question=row['question'],
            response=row['response'],
            context=row['context'],
            ground_truth=row['ground_truth']
        )
        
        # 결과에 메타데이터 추가
        result['agent'] = row['agent']
        result['question_preview'] = row['question'][:50] + '...'
        prompty_results.append(result)
        
        # 진행 상황 표시
        if (idx + 1) % 5 == 0:
            print(f"  [{idx + 1}/{len(df)}] 완료 - 평균 점수: {result['overall_score']:.2f}/5.0")
            
    except Exception as e:
        error_count += 1
        print(f"  [{idx + 1}] ❌ 오류: {str(e)[:80]}")
        continue

# 결과 DataFrame 생성
prompty_df = pd.DataFrame(prompty_results)

# 결과 요약
print(f"\n{'='*60}")
print(f"✓ 평가 완료: 성공 {len(prompty_results)}개, 실패 {error_count}개")
print(f"{'='*60}")

if len(prompty_results) > 0:
    print(f"\n📊 전체 평균 점수:")
    print(f"  종합: {prompty_df['overall_score'].mean():.2f}/5.0")
    print(f"  정확성 (Accuracy): {prompty_df['accuracy_score'].mean():.2f}/5.0")
    print(f"  완전성 (Completeness): {prompty_df['completeness_score'].mean():.2f}/5.0")
    print(f"  관련성 (Relevance): {prompty_df['relevance_score'].mean():.2f}/5.0")
    print(f"  명확성 (Clarity): {prompty_df['clarity_score'].mean():.2f}/5.0")
    
    # 에이전트별 분석
    print("\n📋 에이전트별 평가 결과:\n")
    for agent in sorted(prompty_df['agent'].unique()):
        agent_data = prompty_df[prompty_df['agent'] == agent]
        print(f"  {agent}:")
        print(f"    개수: {len(agent_data)}개")
        print(f"    평균 종합: {agent_data['overall_score'].mean():.2f}/5.0")
        print(f"    정확성: {agent_data['accuracy_score'].mean():.2f}, "
              f"완전성: {agent_data['completeness_score'].mean():.2f}, "
              f"관련성: {agent_data['relevance_score'].mean():.2f}, "
              f"명확성: {agent_data['clarity_score'].mean():.2f}")
        print()
    
    # 상위/하위 3개 샘플
    print("\n🏆 최고 점수 3개 샘플:")
    top3 = prompty_df.nlargest(3, 'overall_score')[['agent', 'question_preview', 'overall_score']]
    for idx, row in top3.iterrows():
        print(f"  {row['overall_score']:.2f}/5.0 - [{row['agent']}] {row['question_preview']}")
    
    print("\n⚠️ 최저 점수 3개 샘플:")
    bottom3 = prompty_df.nsmallest(3, 'overall_score')[['agent', 'question_preview', 'overall_score']]
    for idx, row in bottom3.iterrows():
        print(f"  {row['overall_score']:.2f}/5.0 - [{row['agent']}] {row['question_preview']}")
else:
    print("\n❌ 평가에 성공한 항목이 없습니다.")

=== 전체 데이터셋 Prompty Evaluator 평가 ===
대상: 34개 Q&A 쌍

⚠️ 실제 LLM 호출이 발생합니다:
  - 예상 비용: 약 $0.03 (예상치)
  - 예상 시간: 약 1~2분

진행 중...

  [5/34] 완료 - 평균 점수: 2.75/5.0
  [10/34] 완료 - 평균 점수: 3.50/5.0
  [15/34] 완료 - 평균 점수: 3.50/5.0
  [20/34] 완료 - 평균 점수: 4.75/5.0
  [25/34] 완료 - 평균 점수: 3.25/5.0
  [30/34] 완료 - 평균 점수: 4.25/5.0

✓ 평가 완료: 성공 34개, 실패 0개

📊 전체 평균 점수:
  종합: 3.89/5.0
  정확성 (Accuracy): 3.50/5.0
  완전성 (Completeness): 3.41/5.0
  관련성 (Relevance): 4.12/5.0
  명확성 (Clarity): 4.53/5.0

📋 에이전트별 평가 결과:

  hallucination-agent:
    개수: 7개
    평균 종합: 3.36/5.0
    정확성: 2.00, 완전성: 3.14, 관련성: 4.00, 명확성: 4.29

  policy-agent:
    개수: 6개
    평균 종합: 4.62/5.0
    정확성: 4.50, 완전성: 4.17, 관련성: 5.00, 명확성: 4.83

  product-agent:
    개수: 10개
    평균 종합: 4.00/5.0
    정확성: 3.90, 완전성: 3.60, 관련성: 4.00, 명확성: 4.50

  rag-fail-agent:
    개수: 5개
    평균 종합: 3.05/5.0
    정확성: 2.80, 완전성: 2.20, 관련성: 2.80, 명확성: 4.40

  review-agent:
    개수: 6개
    평균 종합: 4.29/5.0
    정확성: 4.17, 완전성: 3.67, 관련성: 4.67, 명확성: 4.67


🏆 최고 점수 3개 샘플:
  5.00

In [14]:
print("=== Code-based vs Prompty-based 비교 ===\n")

# Code-based 결과 요약
print("📊 Code-based Evaluator (ResponseQualityEvaluator)")
print(f"  평가 항목: {len(quality_df)}개")
print(f"  평균 품질 점수: {quality_df['quality_score'].mean():.2f}")
print(f"  - 길이 점수: {quality_df['length_score'].mean():.2f}")
print(f"  - 구조 점수: {quality_df['structure_score'].mean():.2f}")
print(f"  - 키워드 점수: {quality_df['keyword_score'].mean():.2f}")

# Prompty-based 결과 요약 (있는 경우)
if len(prompty_results) > 0:
    print(f"\n🤖 Prompty-based Evaluator (ResponseQualityPrompyEvaluator)")
    print(f"  평가 항목: {len(prompty_df)}개")
    print(f"  평균 종합 점수: {prompty_df['overall_score'].mean():.2f}/5.0")
    print(f"  - 정확성: {prompty_df['accuracy_score'].mean():.2f}/5.0")
    print(f"  - 완전성: {prompty_df['completeness_score'].mean():.2f}/5.0")
    print(f"  - 관련성: {prompty_df['relevance_score'].mean():.2f}/5.0")
    print(f"  - 명확성: {prompty_df['clarity_score'].mean():.2f}/5.0")

print("\n💡 주요 차이점:")
print("  Code-based:")
print("    + 빠른 실행 속도")
print("    + 비용 없음")
print("    + 객관적이고 일관된 평가")
print("    - 규칙 기반의 제한적 평가")
print("\n  Prompty-based:")
print("    + AI 기반의 정교한 평가")
print("    + Context와 Ground Truth 고려")
print("    + 다차원 평가 (정확성, 완전성, 관련성, 명확성)")
print("    - LLM 비용 발생")
print("    - 느린 실행 속도")

=== Code-based vs Prompty-based 비교 ===

📊 Code-based Evaluator (ResponseQualityEvaluator)
  평가 항목: 34개
  평균 품질 점수: 0.30
  - 길이 점수: 0.61
  - 구조 점수: 0.29
  - 키워드 점수: 0.00

🤖 Prompty-based Evaluator (ResponseQualityPrompyEvaluator)
  평가 항목: 34개
  평균 종합 점수: 3.89/5.0
  - 정확성: 3.50/5.0
  - 완전성: 3.41/5.0
  - 관련성: 4.12/5.0
  - 명확성: 4.53/5.0

💡 주요 차이점:
  Code-based:
    + 빠른 실행 속도
    + 비용 없음
    + 객관적이고 일관된 평가
    - 규칙 기반의 제한적 평가

  Prompty-based:
    + AI 기반의 정교한 평가
    + Context와 Ground Truth 고려
    + 다차원 평가 (정확성, 완전성, 관련성, 명확성)
    - LLM 비용 발생
    - 느린 실행 속도
